In [1]:
from tastytrade import DXLinkStreamer
from tastytrade.dxfeed import Greeks, Quote
from tastytrade import Session, Account
from tastytrade.instruments import Option, Equity, NestedOptionChain, get_option_chain
from tastytrade.order import NewOrder, OrderAction, OrderTimeInForce, OrderType, PlacedOrderResponse

from lib import TTConfig, TTOption, TTOptionSide

from decimal import Decimal
from datetime import date, timedelta

from live_prices import TastytradeWrapper
from strategist import IronCondor, PositionManager
from account_updates import AccountUpdates

In [2]:
config = TTConfig(filename='tt.sandbox.config')
# config = TTConfig(filename='tt.config')
session = Session(config.username, config.password, is_test=not config.use_prod)

In [3]:
account = Account.get_accounts(session)[0]

In [4]:
full_chain = get_option_chain(session, 'SPXW')

In [5]:
options_for_today = full_chain[date.today() + timedelta(days=4)]
options_for_today

[Option(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206C02400000', active=True, strike_price=Decimal('2400.0'), root_symbol='SPXW', underlying_symbol='SPX', expiration_date=datetime.date(2024, 12, 6), exercise_style='European', shares_per_contract=100, option_type=<OptionType.CALL: 'C'>, option_chain_type='Standard', expiration_type='Weekly', settlement_type='PM', stops_trading_at=datetime.datetime(2024, 12, 6, 21, 15, tzinfo=TzInfo(UTC)), market_time_instrument_collection='Cash Settled Equity Option', days_to_expiration=4, expires_at=datetime.datetime(2024, 12, 6, 21, 15, tzinfo=TzInfo(UTC)), is_closing_only=False, streamer_symbol='.SPXW241206C2400', listed_market=None, halted_at=None, old_security_number=None),
 Option(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P02400000', active=True, strike_price=Decimal('2400.0'), root_symbol='SPXW', underlying_symbol='SPX', expiration_date=datetime.date(2024, 12, 6), 

In [6]:
op_pb = Option.get_option(session, 'SPXW  241206P05920000')
op_ps = Option.get_option(session, 'SPXW  241206P05950000')
op_cs = Option.get_option(session, 'SPXW  241206C06060000')
op_cb = Option.get_option(session, 'SPXW  241206C06090000')


In [7]:
position_to_open = IronCondor(op_pb, op_ps, op_cs, op_cb)

In [8]:
account_updates = await AccountUpdates.create(session, account)

In [9]:
position_manager = PositionManager(account_updates, position=position_to_open)

In [10]:
response = await position_manager.open_position(session, account, dry_run=False)
response

PlacedOrderResponse(buying_power_effect=BuyingPowerEffect(change_in_margin_requirement=Decimal('-3000.0'), change_in_buying_power=Decimal('-3002.124'), current_buying_power=Decimal('999989.752'), new_buying_power=Decimal('996987.628'), isolated_order_margin_requirement=Decimal('-3000.0'), is_spread=True, impact=Decimal('3002.124'), effect=<PriceEffect.DEBIT: 'Debit'>), order=PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.ROUTED: 'Routed'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 18, 59, 838000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.BUY_TO_OPEN: 'Buy to Open'>, quantity=Decimal('1'), remaining_quantity=Decimal('1'), fills=[]), Leg(instrument_typ

In [11]:
position_manager.get_open_order()

PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.FILLED: 'Filled'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 18, 59, 956000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.BUY_TO_OPEN: 'Buy to Open'>, quantity=Decimal('1'), remaining_quantity=Decimal('0'), fills=[FillInfo(fill_id='2_TW::TEST_A1::20241202.44-TEST_FILL', quantity=Decimal('1'), fill_price=Decimal('0.01'), filled_at=datetime.datetime(2024, 12, 2, 18, 18, 59, 849000, tzinfo=TzInfo(UTC)), destination_venue='TEST_A', ext_group_fill_id='0', ext_exec_id='43')]), Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05950000', action=<OrderAction.SELL_TO_OPEN: 'Se

In [12]:
account.get_order_history(session)

[PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.FILLED: 'Filled'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 18, 59, 956000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.BUY_TO_OPEN: 'Buy to Open'>, quantity=Decimal('1'), remaining_quantity=Decimal('0'), fills=[FillInfo(fill_id='2_TW::TEST_A1::20241202.44-TEST_FILL', quantity=Decimal('1'), fill_price=Decimal('0.01'), filled_at=datetime.datetime(2024, 12, 2, 18, 18, 59, 849000, tzinfo=TzInfo(UTC)), destination_venue='TEST_A', ext_group_fill_id='0', ext_exec_id='43')]), Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05950000', action=<OrderAction.SELL_TO_OPEN: 'S

In [13]:
position_manager.opening_profit()

Decimal('0.07')

In [14]:
close_response = await position_manager.close_position(session, account, dry_run=False)
close_response

PlacedOrderResponse(buying_power_effect=BuyingPowerEffect(change_in_margin_requirement=Decimal('3000.0'), change_in_buying_power=Decimal('2991.876'), current_buying_power=Decimal('996989.628'), new_buying_power=Decimal('999981.504'), isolated_order_margin_requirement=Decimal('0.0'), is_spread=False, impact=Decimal('2991.876'), effect=<PriceEffect.CREDIT: 'Credit'>), order=PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.ROUTED: 'Routed'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 19, 46, 491000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.SELL_TO_CLOSE: 'Sell to Close'>, quantity=Decimal('1'), remaining_quantity=Decimal('1'), fills=[]), Leg(instrument_ty

In [15]:
position_manager.get_close_order()

PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.FILLED: 'Filled'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 19, 46, 614000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.SELL_TO_CLOSE: 'Sell to Close'>, quantity=Decimal('1'), remaining_quantity=Decimal('0'), fills=[FillInfo(fill_id='2_TW::TEST_A1::20241202.50-TEST_FILL', quantity=Decimal('1'), fill_price=Decimal('0.01'), filled_at=datetime.datetime(2024, 12, 2, 18, 19, 46, 501000, tzinfo=TzInfo(UTC)), destination_venue='TEST_A', ext_group_fill_id='2', ext_exec_id='49')]), Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05950000', action=<OrderAction.BUY_TO_CLOSE:

In [16]:
position_manager.account_updates.orders.keys()

dict_keys([157998, 157999])

In [17]:
account.get_order_history(session)

[PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.DAY: 'Day'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.FILLED: 'Filled'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 12, 2, 18, 19, 46, 614000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05920000', action=<OrderAction.SELL_TO_CLOSE: 'Sell to Close'>, quantity=Decimal('1'), remaining_quantity=Decimal('0'), fills=[FillInfo(fill_id='2_TW::TEST_A1::20241202.50-TEST_FILL', quantity=Decimal('1'), fill_price=Decimal('0.01'), filled_at=datetime.datetime(2024, 12, 2, 18, 19, 46, 501000, tzinfo=TzInfo(UTC)), destination_venue='TEST_A', ext_group_fill_id='2', ext_exec_id='49')]), Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206P05950000', action=<OrderAction.BUY_TO_CLOSE

In [18]:
session.destroy()

## Alternative Path to send order by hand

In [9]:
leg6000 = op_cs.build_leg(Decimal('1'), OrderAction.SELL_TO_OPEN)
leg6030 = op_cb.build_leg(Decimal('1'), OrderAction.BUY_TO_OPEN)

In [10]:
order = NewOrder(
    time_in_force=OrderTimeInForce.GTD,
    order_type=OrderType.LIMIT,
    legs=[leg6000, leg6030],
    price=Decimal('0.05')
    )

In [12]:
order_response = account.place_order(session, order)

In [13]:
order_response

PlacedOrderResponse(buying_power_effect=BuyingPowerEffect(change_in_margin_requirement=Decimal('-3000.0'), change_in_buying_power=Decimal('-2998.562'), current_buying_power=Decimal('1000000.0'), new_buying_power=Decimal('997001.438'), isolated_order_margin_requirement=Decimal('-3000.0'), is_spread=True, impact=Decimal('2998.562'), effect=<PriceEffect.DEBIT: 'Debit'>), order=PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.GTD: 'GTD'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.RECEIVED: 'Received'>, cancellable=True, editable=True, edited=False, updated_at=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241206C06060000', action=<OrderAction.SELL_TO_OPEN: 'Sell to Open'>, quantity=Decimal('1'), remaining_quantity=Decimal('1'), fills=[]), Leg(instrument_type=<Instrumen

In [16]:
account.get_order_history(session)

[PlacedOrder(account_number='5WX24638', time_in_force=<OrderTimeInForce.GTC: 'GTC'>, order_type=<OrderType.LIMIT: 'Limit'>, underlying_symbol='SPX', underlying_instrument_type=<InstrumentType.EQUITY: 'Equity'>, status=<OrderStatus.FILLED: 'Filled'>, cancellable=False, editable=False, edited=False, updated_at=datetime.datetime(2024, 11, 15, 19, 42, 52, 749000, tzinfo=TzInfo(UTC)), legs=[Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241115C06000000', action=<OrderAction.SELL_TO_CLOSE: 'Sell to Close'>, quantity=Decimal('1'), remaining_quantity=Decimal('0'), fills=[FillInfo(fill_id='2_TW::TEST_A1::20241115.197-TEST_FILL', quantity=Decimal('1'), fill_price=Decimal('0.06'), filled_at=datetime.datetime(2024, 11, 15, 19, 42, 52, 647000, tzinfo=TzInfo(UTC)), destination_venue='TEST_A', ext_group_fill_id='1', ext_exec_id='196')]), Leg(instrument_type=<InstrumentType.EQUITY_OPTION: 'Equity Option'>, symbol='SPXW  241115C06100000', action=<OrderAction.BUY_TO_O

In [ ]:
session.destroy()